* Author: Niclas Lavesson
* Date: 2026-01-15
* Description: This notebook runs code that create a composite score for what features to include in the predictive model. Six methods are used to create the composite score
* METHOD 1: Variance Threshold
* METHOD 2: SelectKBest with F-test
* METHOD 3: Mutual Information
* METHOD 4: Random Forest Feature Importance
* METHOD 5: Recursive Feature Elimination (RFE)
* METHOD 6: L1-based Selection (Lasso)

By examining the curve of the composite scores it is possible to see where the number of features (and which) that have less explantory/or predictive power. This information is saved in an xlsx-file.

# Loading data from BQ

In [2]:
from google.cloud import bigquery as bq
import pandas as pd

bq_client = bq.Client()

def load_bq_table_to_dataframe(table_path: str):
    """
    Loads data from a specified BigQuery table into a pandas DataFrame.

    Args:
        table_path (str): The full path to the BigQuery table (e.g., 'project.dataset.table').

    Returns:
        pandas.DataFrame: A DataFrame containing the data from the BigQuery table.
    """
    query_string = f"SELECT * FROM `{table_path}`"
    print(f"Executing BigQuery query for table: {table_path}")
    bq_query_job = bq_client.query(query_string, job_config=bq.QueryJobConfig())
    return bq_query_job.to_dataframe()

# Define the table paths (as strings)
base_path = 'ingka-ff-somdata-prod.OMDA_Analytics'
train_table_path = f'{base_path}.no_stock_prediction_train_data_reduced'
test_table_path = f'{base_path}.no_stock_prediction_test_data_reduced'

# Load data frames
merged_train_data = load_bq_table_to_dataframe(train_table_path)
merged_test_data = load_bq_table_to_dataframe(test_table_path)

print("DataFrames loaded successfully!")

Executing BigQuery query for table: ingka-ff-somdata-prod.OMDA_Analytics.no_stock_prediction_train_data_reduced
Executing BigQuery query for table: ingka-ff-somdata-prod.OMDA_Analytics.no_stock_prediction_test_data_reduced
DataFrames loaded successfully!


## Create X_train, X_test, y_train, y_test

In [3]:
# Define the list of feature columns
feature_cols = [col for col in merged_train_data.columns if col != 'deviation']

# 1. Create y_train from merged_train_data
y_train = merged_train_data['deviation'].copy()
print(f"\ny_train created. Shape: {y_train.shape}")

# 2. Create X_train from merged_train_data
# Ensure all feature_cols exist in merged_train_data
missing_train_cols = [col for col in feature_cols if col not in merged_train_data.columns]
if missing_train_cols:
    print(f"Warning: The following feature columns are missing from merged_train_data: {missing_train_cols}")
    # Filter feature_cols to only include those present in the DataFrame
    X_train_cols = [col for col in feature_cols if col in merged_train_data.columns]
    X_train = merged_train_data[X_train_cols].copy()
else:
    X_train = merged_train_data[feature_cols].copy()
print(f"X_train created. Shape: {X_train.shape}")

# 3. Create y_test from merged_test_data
y_test = merged_test_data['deviation'].copy()
print(f"\ny_test created. Shape: {y_test.shape}")

# 4. Create X_test from merged_test_data
# Ensure all feature_cols exist in merged_test_data
missing_test_cols = [col for col in feature_cols if col not in merged_test_data.columns]
if missing_test_cols:
    print(f"Warning: The following feature columns are missing from merged_test_data: {missing_test_cols}")
    # Filter feature_cols to only include those present in the DataFrame
    X_test_cols = [col for col in feature_cols if col in merged_test_data.columns]
    X_test = merged_test_data[X_test_cols].copy()
else:
    X_test = merged_test_data[feature_cols].copy()
print(f"X_test created. Shape: {X_test.shape}")

# Display the first few rows of the created DataFrames to verify
print("\n--- X_train Head ---")
print(X_train.head())
print("\n--- y_train Head ---")
print(y_train.head())
print("\n--- X_test Head ---")
print(X_test.head())
print("\n--- y_test Head ---")
print(y_test.head())


y_train created. Shape: (1416764,)
X_train created. Shape: (1416764, 125)

y_test created. Shape: (354192,)
X_test created. Shape: (354192, 125)

--- X_train Head ---
   sl3_ordered_qty  average_daily_stock_change_rate  \
0         0.288102                         0.274346   
1        -0.319347                         0.260025   
2        -0.319347                        -1.222145   
3         0.136239                         0.260025   
4         1.047412                         0.328351   

   average_daily_stock_change_rate_replenishment  \
0                                      -0.224212   
1                                      -0.132106   
2                                       0.548648   
3                                       0.398681   
4                                      -0.128225   

   current_daily_stock_change_rate  customer_type_PRIVATE  \
0                        -0.014684                    1.0   
1                         0.222302                    1.0   
2    

# Running feature selection procedure

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import (
    VarianceThreshold,
    SelectKBest,
    f_classif,
    mutual_info_classif,
    RFE,
    SelectFromModel
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("AUTOMATED FEATURE SELECTION PIPELINE")
print("="*80)
print(f"\nInitial number of features: {X_train.shape[1]}")
print(f"Number of training samples: {X_train.shape[0]}")
print(f"Number of test samples: {X_test.shape[0]}")

# Store feature names
feature_names = X_train.columns.tolist()

# Initialize results dictionary
feature_scores = {}

# ==================================================================================
# METHOD 1: Variance Threshold
# ==================================================================================
print("\n" + "="*80)
print("METHOD 1: VARIANCE THRESHOLD")
print("="*80)

selector_var = VarianceThreshold(threshold=0.01)
selector_var.fit(X_train)
var_features = X_train.columns[selector_var.get_support()].tolist()

print(f"Features passing variance threshold: {len(var_features)}/{len(feature_names)}")
feature_scores['variance'] = {feat: 1 if feat in var_features else 0 for feat in feature_names}

# ==================================================================================
# METHOD 2: SelectKBest with F-test
# ==================================================================================
print("\n" + "="*80)
print("METHOD 2: SELECTKBEST (F-TEST)")
print("="*80)

k_best = min(50, X_train.shape[1])
selector_kbest = SelectKBest(f_classif, k=k_best)
selector_kbest.fit(X_train, y_train)

kbest_scores = pd.DataFrame({
    'feature': feature_names,
    'score': selector_kbest.scores_
}).sort_values('score', ascending=False)

print(f"Top 10 features by F-test:")
print(kbest_scores.head(10))

feature_scores['f_test'] = dict(zip(kbest_scores['feature'], kbest_scores['score']))

# ==================================================================================
# METHOD 3: Mutual Information
# ==================================================================================
print("\n" + "="*80)
print("METHOD 3: MUTUAL INFORMATION")
print("="*80)

mi_scores = mutual_info_classif(X_train, y_train, random_state=42)
mi_df = pd.DataFrame({
    'feature': feature_names,
    'score': mi_scores
}).sort_values('score', ascending=False)

print(f"Top 10 features by Mutual Information:")
print(mi_df.head(10))

feature_scores['mutual_info'] = dict(zip(mi_df['feature'], mi_df['score']))

# ==================================================================================
# METHOD 4: Random Forest Feature Importance
# ==================================================================================
print("\n" + "="*80)
print("METHOD 4: RANDOM FOREST FEATURE IMPORTANCE")
print("="*80)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

rf_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"Top 10 features by Random Forest:")
print(rf_importance.head(10))

feature_scores['random_forest'] = dict(zip(rf_importance['feature'], rf_importance['importance']))

# ==================================================================================
# METHOD 5: Recursive Feature Elimination (RFE)
# ==================================================================================
print("\n" + "="*80)
print("METHOD 5: RECURSIVE FEATURE ELIMINATION (RFE)")
print("="*80)

n_features_rfe = min(30, X_train.shape[1])
estimator = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
rfe_selector = RFE(estimator, n_features_to_select=n_features_rfe, step=1)
rfe_selector.fit(X_train, y_train)

rfe_ranking = pd.DataFrame({
    'feature': feature_names,
    'ranking': rfe_selector.ranking_
}).sort_values('ranking')

print(f"Top 10 features by RFE:")
print(rfe_ranking.head(10))

# Convert ranking to score (lower rank = higher score)
max_rank = rfe_ranking['ranking'].max()
feature_scores['rfe'] = {feat: (max_rank - rank + 1) for feat, rank in zip(rfe_ranking['feature'], rfe_ranking['ranking'])}

# ==================================================================================
# METHOD 6: L1-based Selection (Lasso)
# ==================================================================================
print("\n" + "="*80)
print("METHOD 6: L1-BASED SELECTION (LOGISTIC REGRESSION)")
print("="*80)

l1_model = LogisticRegression(penalty='l1', solver='liblinear', C=0.1, random_state=42, max_iter=1000)
l1_selector = SelectFromModel(l1_model, prefit=False)
l1_selector.fit(X_train, y_train)

l1_features = X_train.columns[l1_selector.get_support()].tolist()
print(f"Features selected by L1: {len(l1_features)}/{len(feature_names)}")

feature_scores['l1_selection'] = {feat: 1 if feat in l1_features else 0 for feat in feature_names}

# ==================================================================================
# AGGREGATE RESULTS
# ==================================================================================
print("\n" + "="*80)
print("AGGREGATING RESULTS FROM ALL METHODS")
print("="*80)

# Create a comprehensive DataFrame
results_df = pd.DataFrame(feature_scores).fillna(0)

# Normalize scores to 0-1 range for each method
for col in results_df.columns:
    max_val = results_df[col].max()
    if max_val > 0:
        results_df[col] = results_df[col] / max_val

# Calculate consensus score (average across all methods)
results_df['consensus_score'] = results_df.mean(axis=1)
results_df['feature'] = feature_names
results_df = results_df.sort_values('consensus_score', ascending=False)

print("\nTop 20 features by consensus score:")
print(results_df[['feature', 'consensus_score']].head(20))

# ==================================================================================
# SELECT FINAL FEATURES
# ==================================================================================
print("\n" + "="*80)
print("SELECTING FINAL FEATURES")
print("="*80)

# You can adjust top_n to select how many features you want
top_n = 30  # Adjust this number based on your needs
selected_features = results_df.head(top_n)['feature'].tolist()

print(f"\nSelected {len(selected_features)} features:")
for i, feat in enumerate(selected_features, 1):
    score = results_df[results_df['feature'] == feat]['consensus_score'].values[0]
    print(f"  {i}. {feat}: {score:.4f}")

# ==================================================================================
# TRANSFORM DATASETS
# ==================================================================================
print("\n" + "="*80)
print("TRANSFORMING DATASETS")
print("="*80)

X_train_selected = X_train[selected_features].copy()
X_test_selected = X_test[selected_features].copy()

print(f"\nOriginal X_train shape: {X_train.shape}")
print(f"Selected X_train shape: {X_train_selected.shape}")
print(f"Original X_test shape: {X_test.shape}")
print(f"Selected X_test shape: {X_test_selected.shape}")

# ==================================================================================
# SAVE RESULTS
# ==================================================================================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

output_filename = 'feature_selection_comprehensive_results.xlsx'
output_path = f'C:/Users/NILAV/OneDrive - IKEA/Documents/Project folder/NO STOCK deviation predictions/Feature engineering/{output_filename}'

# Reorder columns for better readability
cols_order = ['feature', 'consensus_score'] + [col for col in results_df.columns if col not in ['feature', 'consensus_score']]
results_df_export = results_df[cols_order]

results_df_export.to_excel(output_path, index=False)
print(f"\nResults saved to '{output_filename}'")

print("\n" + "="*80)
print("✅ AUTOMATED FEATURE SELECTION COMPLETE!")
print("="*80)
print(f"\nYou can now use X_train_selected and X_test_selected for model training.")
print(f"Selected features are stored in the 'selected_features' list.")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(range(1, len(results_df) + 1), results_df['consensus_score'].values, marker='o')
plt.xlabel('Feature Rank')
plt.ylabel('Consensus Score')
plt.title('Consensus Score by Feature Rank (Look for the "Elbow")')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.5, color='r', linestyle='--', label='Example threshold: 0.5')
plt.axhline(y=0.3, color='orange', linestyle='--', label='Example threshold: 0.3')
plt.legend()
plt.tight_layout()
plt.show()
